In [1]:
import numpy as np
from scipy.stats import laplace

In [2]:
np.random.seed(2786345)

# # Specify these four parameters
p_zero = 0.13
N = 2000
delta_bleu = 1.0
b_zero = 25.8

n_trials = 500  # number of outer loops
n_randomizations = 1000   # number of inner loops

# compute mu (Laplace location parameter) and b (Laplace scale parameter)
mu = -2 * delta_bleu / (N * (1-p_zero))
b = b_zero / N

uppers = []
all_diffs = []
sig_diffs = []
n_successes = 0

for i in range(n_trials):
    # draw a sample of individual effects
    laplace_samples = np.random.laplace(mu, b, size=N)
    # zero out a random proportion of them
    zeros = np.random.choice([0,1], size=N, p=[p_zero, 1-p_zero])
    samples = zeros*laplace_samples

    # determine what the implied difference between models is for these swaps
    # it should be half the total change that would result if we flipped everything
    observed_difference = -np.sum(samples)/2
    all_diffs.append(observed_difference)

    # choose random subsets to apply and then aggregate
    effects = []
    for j in range(n_randomizations):
        # get a list of random floats
        rands = np.random.rand(N)
        # with p=0.5, choose an item (i.e. make that flip)
        sel = np.array(rands > 0.5, dtype=float)
        # compute the (approximate) net effect of making all those swaps
        total_effect = np.sum(samples * sel)
        effects.append(total_effect)

    # apply these effects to get the null distribution
    null_vals = np.array(effects) + observed_difference

    # assuming our initial difference is positive, look at the upper tail
    upper = np.percentile(null_vals, q=97.5)  
    lower = np.percentile(null_vals, q=2.5)
    uppers.append(upper)
    
    # if the original difference for this sample is above this threshold, that's significant  
    if observed_difference >= upper or observed_difference <= lower:
        sig_diffs.append(observed_difference)
        if np.sign(observed_difference) == np.sign(delta_bleu):
            n_successes += 1
    
power = n_successes / n_trials
print("Mean observed difference = {:.3f}".format(np.mean(all_diffs)))
print("Approximate power = {:.3f}".format(power))
print("Approximate Type-M error = {:.3f}".format(np.mean(np.abs(sig_diffs))/np.abs(delta_bleu)))
print("Approximate Type-S error = {:.3f}".format(np.mean(np.sign(sig_diffs) != np.sign(delta_bleu))))


Mean observed difference = 1.026
Approximate power = 0.756
Approximate Type-M error = 1.200
Approximate Type-S error = 0.000
